In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.metrics import label_ranking_average_precision_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM, GRU, Reshape, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.data import Dataset
import tensorflow as tf

from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow_addons.metrics import HammingLoss, F1Score, hamming_loss_fn, hamming_distance

import datetime
import sys

In [2]:
def make_model(opt, ls, i_features, o_features, i_len, o_len, metrics):
    
    x = Input(batch_shape=(None, i_len, i_features), name='INPUT')
    lstm1 = GRU(50, name='LSTM1', return_sequences=True)(x)
    lstm1 = GRU(50, name='LSTM2')(lstm1)
    repeater = RepeatVector(o_len, name='Repeater')(lstm1)
    lstm2 = GRU(50, name='LSTM3', return_sequences=True)(repeater)    
    lstm2 = GRU(50, name='LSTM4', return_sequences=True)(lstm2)    
    y = TimeDistributed(Dense(o_features, activation='sigmoid', name='OUT'))(lstm2)
    
    model = Model(x, y, name=f'Seq2Seq-{i_len}-{o_len}')
    model.compile(loss=ls, optimizer=opt, metrics=metrics)
    return model

In [3]:
def Custom_Hamming_Loss(y_true, y_pred):
    return tf.math.reduce_mean(tf.cast(tf.math.greater(tf.math.abs(y_true-y_pred),0.5),dtype=float))

In [4]:
def train_results(inp, out):
    with np.load(f'arrays/np_input_{inp}_{out}.npz') as data:
        inp_data = data['arr_0']
    with np.load(f'arrays/np_output_{inp}_{out}.npz') as data:
        out_data = data['arr_0']
        out_data = out_data
    
    train_x, test_x, train_y, test_y = train_test_split(inp_data, out_data, test_size=0.2)    
    
    ep = 100
    i_len = train_x[0].shape[0]
    o_len = train_y[0].shape[0]
    inp_features = train_x[0].shape[-1]
    out_features = train_y[0].shape[-1]
    lr = 0.001
    decay = 0.001
    batch = 5120
    opt = Adamax(lr=lr, decay=decay)
    ls = tf.nn.sigmoid_cross_entropy_with_logits
    metrics = [ 'categorical_accuracy', 'Precision', 'Recall', HammingLoss(mode='multilabel')]
    
    filepath=f'tf/checkpoints/model-{inp}-{out}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_freq=10)

    log_dir = f'tf/logs/fit/model-{inp}-{out}'
    tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

    callbacks_list = [checkpoint, tensorboard]
    
    model = make_model(opt, ls, inp_features, out_features, i_len, o_len, metrics)
    h = model.fit(x=train_x, y=train_y, 
              validation_data=(test_x, test_y),
              epochs=ep, 
              shuffle=True,
              batch_size=batch,
              callbacks=callbacks_list,
              verbose=0)

    model.save(f'tf/models/model-{inp}-{out}.h5' )
    print(f'Model {inp}-{out}:')
    model.evaluate(x=test_x, y=test_y)


In [ ]:
in_len_array = [3, 3, 3, 5, 5, 5, 10, 10, 10]
out_len_array = [1, 2, 3, 1, 3, 5, 1, 3, 5]

for inp, out in zip(in_len_array,out_len_array ):
    train_results(inp, out)